In order for this to run you need to install energyPlus in the DEFAULT location - I'm not sure if this is a requirement for eppy, but I couldn't find any other way <br>
To install on linux I simply uploaded version the".sh" file to my home dir <br>
then changed the file permissions so it could run, i.e. command: "chmod +x *filename*"  <br>
Then ran the installation .sh file 

In [1]:
import numpy as np
import glob
import os
import cPickle as pickle
from collections import defaultdict
import sys
import numpy as np
import shutil
from shutil import copyfile

# eppy paramters
from eppy import modeleditor
from eppy.modeleditor import IDF

In [2]:
# set the paths to your own idd files and the weather file (epwFile)
iddfile = './Energy+.idd'
IDF.setiddname(iddfile)
# I found the weather file path had to be absolute
epwFile = './BOS.epw'#'whatever the path to BOS.epw'

In [3]:
# Depending on what simulations are being run, comment/uncomment an inDir-outDir combo

inDir = 'All_buildings_standard_efficiency'
outDir = 'Standard_efficiency'

# inDir = 'Apartment_insulation_increase'
# outDir ='Ins_upgrade'

# inDir = 'Commercial_equipment_efficiency'
# outDir = 'Eqp_upgrade'

listFiles = glob.glob('../Idf_generated_files/'+inDir+'/*.idf')

In [4]:
print 'About to run', len(listFiles), 'simulations'

About to run 954 simulations


In [5]:
problemBees = []
counter = 0
cwdPATH = os.getcwd()
for f in listFiles:
    counter+=1
    if counter%100==0:
        print counter
    idf1 = IDF(f,epwFile)
    outName = f.split(inDir+'/')[1].split('.')[0]
    try:
        idf1.run(output_directory=u'../EPS_Results/'+outDir, output_prefix=outName, readvars=True)
    except:
        problemBees.append(f)
        # if the command doesnt work it stays in the temp dir, therefore change it back using ABSOLUTE path
        os.chdir(cwdPATH)
    dontDelete=glob.glob('../EPS_Results/'+outDir+'/*out.csv')
    gitFiles=glob.glob('../EPS_Results/'+outDir+'/.gitignore')
    listDelete = list(set(glob.glob('../EPS_Results/'+outDir+'/*'))-set(dontDelete)-set(gitFiles))
    for file2 in listDelete:
        os.remove(file2)

100
200
300
400
500
600
700
800
900


In [16]:
# now rename all the files so as not to include out
outFileList = glob.glob('../EPS_Results/'+outDir+'/*.csv')

In [7]:
for f in outFileList:
    newName = f.split(outDir+'/')[1].split('out.')[0]
    newName = '../EPS_Results/'+outDir+'/'+newName+'.csv'
    os.rename(f,newName)

In [8]:
## save the problem building ids
with open('../Intermediate_Results/problemBuildingsList.pickle', 'wb') as handle:
    pickle.dump(problemBees, handle, pickle.HIGHEST_PROTOCOL)

In [17]:
problemBees

['../Idf_generated_files/All_buildings_standard_efficiency/0501354000_experiment1.idf']

In [24]:
len(glob.glob('../EPS_Results/Standard_efficiency/*experiment1.csv'))

318

In [9]:
# os.chdir(cwdPATH)

In [18]:
# name = '../Idf_generated_files/All_buildings_standard_efficiency/0501354000_experiment1.idf'
# outName = name.split(inDir+'/')[1].split('.')[0]
# idf1 = IDF(name,epwFile)
# idf1.run(output_directory=u'../testings', output_prefix=outName, readvars=True)

# os.chdir(cwdPATH)

In [11]:
# fileList = glob.glob('../Full_idf_files/*_0.idf')
# print len(fileList)
# for f in fileList:
#     if '_0' not in f:
#         print f

In [12]:
# fileList[0]

In [13]:
# idList = []
# for f in problemBees:
#     if 'control' in f:
#         idList.append( f.split('efficiency/')[1].split('_')[0]+'_0.idf' )

In [14]:
# len(idList)

In [15]:
# ### go through problemBees and change heating/cooling setpoints

# for idL in idList:
#     count1, count2 = 0, 0
#     print '../Full_idf_files/'+idL
#     idf1 = IDF('../Full_idf_files/'+idL)
#     schedules = idf1.idfobjects['SCHEDULE:COMPACT']
#     for s in schedules:
#         if 'Heating' in s.Name:
#             count1+=1
#             heatVal = np.float( s['obj'][6] )
#             #print heatVal, s.Name
#             coolSchedName = s.Name
#             coolSchedName = coolSchedName.replace('Heating', 'Cooling')
#             coolSched = [sch for sch in schedules if sch.Name==coolSchedName][0]
#             coolVal = np.float( coolSched['obj'][6] )
#             #print coolVal, coolSchedName
#             # if heatVal greater than coolVal, switch the schedules
#             if heatVal>coolVal:
#                 count2+=1
#                 dummySched = s['obj'][:]
#                 for j in range(2,len(s['obj'])):
#                     s['obj'][j] = coolSched['obj'][j]
#                     coolSched['obj'][j] = dummySched[j]
    
#     print 'Building %s'%idL+' has %d'%count2+' problems out of %d'%count1                 
#     idf1.save()